<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/AnswerClusterer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   load generate answers
2.   Entailment model load //trin
3.   Clustering useing the ent model
4.   Save the output into a file





# set ups

In [1]:
!huggingface-cli login
!pip install datasets
!pip install  transformers torch evaluate


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) ى
Invalid input. Must be one of ('y', 'yes', '1', 'n', 'no', '0', '')
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `week1 test` has been saved to /root/.cache/huggingface/stored_tokens
Your token

In [2]:
import torch
import json
import pandas as pd
import os
import torch.nn.functional as F
from transformers import (
    ElectraForSequenceClassification,
    ElectraTokenizerFast,
    Trainer,
    TrainingArguments,
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification
)
from datasets import load_dataset
import numpy as np
from evaluate import load
import json
from google.colab import files

# load and fine-tuned the  model

In [ ]:
#load functions

# Load XNLI dataset
print("Loading XNLI dataset...")
dataset = load_dataset("xnli", "ar")

# Load AraELECTRA model and tokenizer
model_name = "aubmindlab/araelectra-base-discriminator"
tokenizer = ElectraTokenizerFast.from_pretrained(model_name)
model = ElectraForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3  # entailment, neutral, contradiction
)



Loading XNLI dataset...


train-00000-of-00001.parquet:   0%|          | 0.00/58.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/392k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/194k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at aubmindlab/araelectra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#trinning functions
# Preprocess function with labels
def preprocess_function(examples):

    # Tokenize the premise and hypothesis pairs
    tokenized = tokenizer(
        examples["premise"],
        examples["hypothesis"],
        truncation=True,
        max_length=128,
        padding="max_length"
    )

    # Add labels to the tokenized output
    tokenized["labels"] = examples["label"]

    return tokenized

# Tokenize datasets
print("Tokenizing datasets...")
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

# Print sample to verify data format
print("\nSample preprocessed data:")
print(tokenized_datasets["train"][0])

# Load accuracy metric
metric = load("accuracy")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./araelectra-nli-finetuned",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="wandb",
    logging_dir="./logs",
    logging_steps=1000,
    fp16=True  # Enable mixed precision training
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,         # Add tokenizer to trainer
)


# Fine-tune the model
print("\nStarting training...")
train_result = trainer.train()

# Print training results
print("\nTraining results:")
print(train_result)

# Save the fine-tuned model
model_save_path = "./araelectra-nli-finetuned"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"\nModel saved to {model_save_path}")



Tokenizing datasets...


Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/5010 [00:00<?, ? examples/s]

Map:   0%|          | 0/2490 [00:00<?, ? examples/s]


Sample preprocessed data:
{'input_ids': [2, 19, 139, 602, 298, 7249, 13782, 300, 11936, 59778, 19, 8222, 43907, 20, 3, 8222, 43907, 583, 394, 3546, 7249, 38749, 192, 700, 20, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: altwujaeire-reema (altwujaeire-reema-king-saud-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.548900,0.585303,0.773092
2,0.428800,0.636830,0.773896
3,0.313600,0.750802,0.769880



Training results:
TrainOutput(global_step=73632, training_loss=0.4573977962362927, metrics={'train_runtime': 4509.9684, 'train_samples_per_second': 261.223, 'train_steps_per_second': 16.327, 'total_flos': 7.74938740264658e+16, 'train_loss': 0.4573977962362927, 'epoch': 3.0})

Model saved to ./araelectra-nli-finetuned


In [ ]:
#used this to download the fine-tuned model after training
# Zip the model directory to make it easier to download
!zip -r araelectra-nli-finetuned.zip araelectra-nli-finetuned/

# This will create a download link in Colab
from google.colab import files
files.download('araelectra-nli-finetuned.zip')
#get the finetuned model from Reema its too big for GitHub
model = ElectraForSequenceClassification.from_pretrained("./araelectra-nli-finetuned")
tokenizer = ElectraTokenizerFast.from_pretrained("./araelectra-nli-finetuned")


In [3]:
#ues our trained fine-tunied model
import zipfile
import os
from google.colab import drive

#get the finetuned model from drive
drive.mount('/content/drive')

zip_path = "/content/drive/My Drive/araelectra-nli-finetuned.zip"  # Adjust the path if needed
extract_path = "/content/araelectra"

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete.")


Mounted at /content/drive
Extraction complete.


In [22]:
#RUN THIS
#get the finetuned model from Reema its too big for GitHub
model = ElectraForSequenceClassification.from_pretrained("/content/araelectra/araelectra-nli-finetuned")
tokenizer = ElectraTokenizerFast.from_pretrained("/content/araelectra/araelectra-nli-finetuned")

In [5]:
# Test the fine-tuned model
def test_model(premise, hypothesis):
    inputs = tokenizer(
        premise,
        hypothesis,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(predictions, dim=-1).item()

    label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
    return label_map[predicted_class], predictions[0].tolist()

# Test examples
test_examples = [
    ("القاهره عاصمه مصر", "القاهره هي عاصمة دولة مصر"),
    ("القاهره عاصمه مصر", "القاهره ليست عاصمة مصر"),
    ("القاهره عاصمه مصر", "مصر دولة عربية")
]

print("\nTesting fine-tuned model:")
for premise, hypothesis in test_examples:
    prediction, probabilities = test_model(premise, hypothesis)
    print(f"\nPremise: {premise}")
    print(f"Hypothesis: {hypothesis}")
    print(f"Prediction: {prediction}")
    print(f"Probabilities: entailment={probabilities[0]:.3f}, neutral={probabilities[1]:.3f}, contradiction={probabilities[2]:.3f}")


Testing fine-tuned model:

Premise: القاهره عاصمه مصر
Hypothesis: القاهره هي عاصمة دولة مصر
Prediction: entailment
Probabilities: entailment=0.958, neutral=0.039, contradiction=0.004

Premise: القاهره عاصمه مصر
Hypothesis: القاهره ليست عاصمة مصر
Prediction: contradiction
Probabilities: entailment=0.001, neutral=0.009, contradiction=0.989

Premise: القاهره عاصمه مصر
Hypothesis: مصر دولة عربية
Prediction: neutral
Probabilities: entailment=0.048, neutral=0.920, contradiction=0.032


# load  the generation file

In [6]:
#load generation file
file_path = '/content/generated_answers_Llama3.1-8bxor_tydiqa.json'

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.DataFrame(data)
print(df.head())

            question_id                               question  \
0   5434985093263686694              ما هي أولى جامعات فنلندا؟   
1   5727312019662110019   ما عدد الدول المطلة على بحر البلطيق؟   
2  -7900394637934786233                         اين عاش نيوتن؟   
3  -2275140940114918987                هل زار ابن بطوطة اليمن؟   
4  -7366545647565275586  من هو الرئيس الأول للجمهورية اليمنية؟   

                                     original_answer  \
0                                     أكاديمية توركو   
1  فنلندا والسويد والدنمارك وإستونيا ولاتفيا وليت...   
2                                            غرانثام   
3                                                yes   
4                                  علي عبد الله صالح   

                                   generated_answers  
0  [{'text': 'أول جامعة في فنلندا هي جامعة هلسنكي...  
1  [{'text': '9 países', 'neg_log_likelihood': 43...  
2  [{'text': 'عاش في', 'neg_log_likelihood': 16.4...  
3  [{'text': 'لم يذكر ابن بطوطة تفاصيل عن زيار

# entailment functions

In [56]:
import random
def check_entailment_arabic(sentence1, sentence2):
    # Load the fine-tuned model

    model = ElectraForSequenceClassification.from_pretrained("/content/araelectra/araelectra-nli-finetuned")
    tokenizer = ElectraTokenizerFast.from_pretrained("/content/araelectra/araelectra-nli-finetuned")

    # Move to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Prepare input
    inputs = tokenizer(
        sentence1,
        sentence2,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    # Move inputs to device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Get prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=1)
        predicted_class = torch.argmax(probs, dim=1).item()

    # Map prediction to label
    return "entailment" if predicted_class == 0 else "neutral" if predicted_class == 1 else "contradiction" if predicted_class == 2 else "unknown"

def bidirectional_entailment( answer1, answer2):
    entailment1 = check_entailment_arabic(answer1, answer2)
    entailment2 = check_entailment_arabic(answer2, answer1)
    return entailment1 == "entailment" and entailment2 == "entailment"

# Clusters generated answers for a specific question based on bidirectional entailment.
def process_question(question_id, generated_answers):
    clusters = []  # List of clusters
    cluster_likelihoods = []  # List to store total likelihoods for each cluster

   # Iterate over all generated answers
    for answer_data in generated_answers:
        answer = answer_data["text"]
        neg_ll = answer_data["neg_log_likelihood"]
        added_to_cluster = False # Track whether the answer is added to a cluster

      # Compare the answer with representatives of existing clusters
        for i, cluster in enumerate(clusters):
            representative = cluster[0][0] # Take the first answer as representative
            if bidirectional_entailment( representative, answer):
                cluster.append([answer, neg_ll])
                cluster_likelihoods[i] += neg_ll # Add to the cluster's total likelihood
                added_to_cluster = True
                break
        # If the answer does not fit into any cluster, create a new cluster
        if not added_to_cluster:
            clusters.append([[answer, neg_ll]])
            cluster_likelihoods.append(neg_ll)  # Initialize this cluster's likelihood

    return clusters, cluster_likelihoods


# Processes the dataset for entailment clustering
def process_dataset(data):
    results = {}

    for question in data:
        question_id = question["question_id"]
        question_text = question["question"]
        original_answer = question["original_answer"]
        generated_answers = question["generated_answers"]

        print(f"\nProcessing question {question_id}: {question_text}")

        # Generate clusters and get total likelihoods
        clusters, cluster_likelihoods = process_question(question_text, generated_answers)

        #  output structure
        question_result = {
            "question": question_text,
            "original_answer": original_answer,
            "clusters": []
        }

        # Add clusters with their total likelihoods to the result
        for i, cluster in enumerate(clusters , start=0):
            cluster_with_likelihood = {
                "cluster_number": i,  # Add cluster number
                "answers" : cluster,
                "total_neg_log_likelihoods_for_cluster": cluster_likelihoods[i]
            }
            question_result["clusters"].append(cluster_with_likelihood)

        results[question_id] = question_result

    return results

#  Apply the clustering and save the clusters file

In [57]:
# Step 3: Clustering
# Apply the clustering to the loaded dataset
print("\nApplying clustering to the loaded dataset...")


entailment_clusters = process_dataset(data)

# Save results to JSON file
output_file = "entailment_clusters_Llama3.1-8b-xor_tydiqa.json"

with open(output_file, 'w', encoding='utf-8') as file:
    json.dump(entailment_clusters, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file}")

files.download(output_file)



Applying clustering to the loaded dataset...

Processing question 5434985093263686694: ما هي أولى جامعات فنلندا؟

Processing question 5727312019662110019: ما عدد الدول المطلة على بحر البلطيق؟

Processing question -7900394637934786233: اين عاش نيوتن؟

Processing question -2275140940114918987: هل زار ابن بطوطة اليمن؟

Processing question -7366545647565275586: من هو الرئيس الأول للجمهورية اليمنية؟

Processing question -1913760048645526759: متى إنتهت حرب نورماندي ؟

Processing question 4229319161750877372: من هو المستكشف الذي عثرعلى جزيرة القيامة في المحيط الهادي الجنوبي؟

Processing question 7671792355672899855: ماهى أشهر مؤلفات ماري وولستونكرافت ؟

Processing question 6432367568304983636: أين  نزل  القرآن؟

Processing question 4031278892321015014: لماذا يعتبر نظام التعليم الفلندي من افضل الانظمة التعليمية؟

Processing question -8805815200414279194: ما هو أ قدم  فريق كرة قدم في فرنسا؟

Processing question 6856068782797688979: في عام توفي المهاتما غاندي؟

Processing question -662326672233

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>